<a href="https://colab.research.google.com/github/Wincci77/Machine-Learning-with-Python/blob/main/Neural%20Network%20SMS%20Text%20Classifier/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Load data
train_data = pd.read_csv(train_file_path, sep='\t', names=["label", "message"], header=None)
test_data = pd.read_csv(test_file_path, sep='\t', names=["label", "message"], header=None)

In [ ]:
# Ensure data is properly loaded as DataFrame
if not isinstance(train_data, pd.DataFrame) or not isinstance(test_data, pd.DataFrame):
    raise ValueError("train_data and test_data should be DataFrame objects.")

In [ ]:
# Convert labels to binary values
y_train = train_data["label"].map({"ham": 0, "spam": 1})
y_test = test_data["label"].map({"ham": 0, "spam": 1})

X_train, X_test = train_data["message"], test_data["message"]

In [ ]:
# Build text classification model
model = make_pipeline(TfidfVectorizer(ngram_range=(1, 2), stop_words='english', max_df=0.9, min_df=5), MultinomialNB(alpha=0.1))
model.fit(X_train, y_train)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(message):
    """Predict whether a message is spam or ham."""
    prob = model.predict_proba([message])[0][1]  # Get spam probability
    label = "spam" if prob > 0.5 else "ham"
    return [prob, label]

# Test model accuracy
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model accuracy: {accuracy:.2f}")

pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
